# Importing the Modules

In [12]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, LeakyReLU, Dropout, Flatten, Dense, BatchNormalization, Reshape, Conv2DTranspose, Rescaling
from tensorflow.keras import Sequential, preprocessing
import matplotlib.pyplot as plt
import numpy as np
import os
import time

# Data Preperation

In [2]:
train_directory = 'img_align_celeba'
BATCH_SIZE = 64
INPUT_DIM = 64
dataset = tf.keras.preprocessing.image_dataset_from_directory(train_directory, label_mode= None, image_size=(INPUT_DIM, INPUT_DIM), batch_size=BATCH_SIZE)

Found 202599 files belonging to 1 classes.


# Modelling

In [6]:
INIT_SIZE = INPUT_DIM // 4
NOISE_DIM = 128

inputs = tf.keras.Input(shape = (NOISE_DIM, ))

generator_model = Sequential([
    inputs,
    Dense(INIT_SIZE * INIT_SIZE * NOISE_DIM,),

    BatchNormalization(),
    LeakyReLU(),
    Reshape((INIT_SIZE, INIT_SIZE, 128)),
    Conv2DTranspose(128, (5, 5), strides =(1, 1), padding = 'same'),
    Conv2DTranspose(INIT_SIZE * 4, (5, 5), strides =(2, 2), padding = 'same'),

    BatchNormalization(),
    LeakyReLU(),
    Conv2DTranspose(3, (5, 5), strides =(2, 2), padding = 'same', activation = 'tanh'),
])

generator_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 32768)             4227072   
                                                                 
 batch_normalization_1 (Batc  (None, 32768)            131072    
 hNormalization)                                                 
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 32768)             0         
                                                                 
 reshape (Reshape)           (None, 16, 16, 128)       0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 16, 16, 128)      409728    
 nspose)                                                         
                                                                 
 conv2d_transpose_1 (Conv2DT  (None, 32, 32, 64)       2

# Discriminator

In [13]:
inputs = tf.keras.Input(shape = (INPUT_DIM, INPUT_DIM, 3))

discriminator_model = Sequential([
    inputs,
    Rescaling(1 / 255.0),
    Conv2D(32, (5, 5), strides = (2, 2), padding = 'same'),
    LeakyReLU(),
    Dropout(0.3),
    Conv2D(128, (5, 5), strides = (2, 2), padding = 'same'),
    LeakyReLU(),
    Dropout(0.3),
    Conv2D(128, (5, 5), strides = (2, 2), padding = 'same'),
    LeakyReLU(),
    Dropout(0.3),
    Flatten(),
    Dense(1),
    ])
discriminator_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 64, 64, 3)         0         
                                                                 
 conv2d (Conv2D)             (None, 32, 32, 32)        2432      
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 32, 32, 32)        0         
                                                                 
 dropout (Dropout)           (None, 32, 32, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 128)       102528    
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 16, 16, 128)       0         
                                                                 
 dropout_1 (Dropout)         (None, 16, 16, 128)      